In [ ]:
import requests
from bs4 import BeautifulSoup
import ffmpy3 # 多媒体视频处理工具 ffmpeg
from multiprocessing.dummy import Pool as ThreadPool

In [ ]:
search_url = 'https://www.okzyw.com/index.php'
server = 'https://www.okzyw.com/'
search_keyword = '越狱第一季'
search_parameters = {
    'm': 'vod-search'
}
search_headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0',
    'Referer': 'https://www.okzyw.com/',
    'Origin': 'https://www.okzyw.com',
    'Host': 'www.okzyw.com'
}
search_datas = {
    'wd': search_keyword,
    'submit': 'search'
}

# send post request
req = requests.post(url=search_url,
                   params=search_parameters,
                   headers=search_headers,
                   data=search_datas)
req.encoding='utf-8'
search_html = BeautifulSoup(req.text, 'lxml')
search_spans = search_html.find_all('span', class_='xing_vb4')

search_results = []
for span in search_spans:
    url = server + span.a.get('href')
    name = span.a.string
    search_results.append((name, url))


In [ ]:
for name, url in search_results:
    req = requests.get(url)
    req.encoding = 'utf-8'
    details_html = BeautifulSoup(req.text, 'lxml')
    # 所有的集数和url都包含在<li>中
    items = details_html.find_all('li', class_='')
    
    chapters = []
    for item in items:
        # 选择m3u8格式的链接源
        if 'm3u8' in item.text:
            chapter_nam, chapter_url = item.text.split('$')
            chapters.append((chapter_nam, chapter_url))
            


In [ ]:
def get_video(chapters):
    for chapter in chapters:
        print(chapter[0], chapter[1])
        try:
            ffmpy3.FFmpeg(inputs={chapter[1]: None},
                          outputs={chapter[0]: None}).run()
        except Exception as e:
            print("Failed: " + chapter_num + ": " + "chapter_url")
            print(e)
    return 0

# get_video(chapters)

In [ ]:
# 开12个线程池
pool = ThreadPool(12)
results = pool.map(get_video, chapters)
pool.close()
pool.join()